In [1]:
# !wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/tatabahasabm.tripod.com/quiz-tatabahasa.jsonl

In [2]:
from tqdm import tqdm
import os
import json

In [3]:
instructions = []
with open('quiz-tatabahasa.jsonl') as fopen:
    for no, l in enumerate(fopen):
        l = json.loads(l)
        soalan = [l['question']]
        jawapan = None
        for c, k in l['choices'].items():
            soalan.append(f"{c}. {k['text']}")
            if k['answer']:
                jawapan = c
        
        i = 'Jawab soalan yang diberikan' if l['instruction'] is None else l['instruction']
        q = '\n'.join(soalan)
        s = f"{i}\n{q}\nJawapan: {jawapan}"
        instructions.append(s)
        
len(instructions)

349

In [4]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai2-aiservices.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
!mkdir tatabahasa-v2

In [6]:
from tqdm import tqdm
import os

for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasa-v2/{i}.json'
    if os.path.exists(filename):
        continue
        
    r = []
    for k in range(3):
        
        try:
            message_text = [
                {"role":"user","content": f"{instructions[i]}\n\ngenerate similar questions included answers like above."},
            ]
            completion = openai.ChatCompletion.create(
              engine="gpt-4",
              messages = message_text,
              temperature=1.0,
              max_tokens=1024,
              top_p=0.95,
              frequency_penalty=0,
              presence_penalty=0,
              stop=None
            )
            r.append(completion.choices[0]['message']['content'])
            

        except Exception as e:
            print(e)
        
    with open(filename, 'w') as fopen:
        json.dump(r, fopen)

 48%|█████████████████▉                   | 169/349 [1:31:36<1:24:16, 28.09s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 62%|██████████████████████▉              | 216/349 [1:56:38<1:08:35, 30.94s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 62%|███████████████████████              | 217/349 [1:57:15<1:12:32, 32.97s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766


 76%|█████████████████████████████▌         | 264/349 [2:29:29<50:53, 35.92s/it]

'content'


100%|███████████████████████████████████████| 349/349 [3:19:32<00:00, 34.30s/it]


In [9]:
augmentation = []
for i in tqdm(range(0, len(instructions), 1)):
    filename = f'tatabahasa-v2/{i}.json'
    try:
        with open(filename) as fopen:
            datas = json.load(fopen)
        for data in datas:
            splitted = data.split('\n\n')
            for s in splitted:
                a = s.split('Jawapan:')[1].strip()
                augmentation.append({
                    'question': s.split('Jawapan:')[0].strip(),
                    'answer': a,

                })
    except:
        pass

100%|██████████████████████████████████████| 349/349 [00:00<00:00, 13364.85it/s]


In [10]:
len(augmentation)

5979

In [11]:
augmentation[-1]

{'question': '5. Dia mengemudi dengan ........ untuk menghindari ........ kecelakaan.\nA. sembrono-sembrono, terjadi-terjadi\nB. hati-hati, terlibat-terlibat\nC. ugal-ugalan, menyebabkan-menyebabkan\nD. ceroboh-ceroboh, menimbulkan-menimbulkan',
 'answer': 'B'}

In [12]:
with open('synthetic-tatabahasa-v2.jsonl', 'w') as fopen:
    for a in augmentation:
        fopen.write(f'{json.dumps(a)}\n')